In [1]:
import pandas as pd
import boto3
import sagemaker
import json
import joblib
import xgboost as xgb
from sklearn.metrics import roc_auc_score
import numpy as np
import pyarrow
from math import radians, sin, cos, sqrt, atan2
import time
import io
from io import StringIO
import base64
import pprint
import re
from sagemaker.image_uris import retrieve

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
from scipy.stats import norm
import statsmodels.api as sm

In [3]:
pip install s3fs

  Using cached fsspec-2024.3.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached botocore-1.31.64-py3-none-any.whl.metadata (6.1 kB)
Using cached fsspec-2024.3.1-py3-none-any.whl (171 kB)
Using cached botocore-1.31.64-py3-none-any.whl (11.3 MB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.6.0
ERROR: Cannot uninstall fsspec 2023.6.0, RECORD file not found. You might be able to recover from this via: 'pip install --force-reinstall --no-deps fsspec==2023.6.0'.
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pdpbox

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [7]:
sess = sagemaker.Session()
write_bucket = sess.default_bucket()
write_prefix = "search-algorithm-demo"

region = sess.boto_region_name
s3_client = boto3.client("s3", region_name=region)
sm_client = boto3.client("sagemaker", region_name=region)
sm_runtime_client = boto3.client("sagemaker-runtime")
sm_autoscaling_client = boto3.client("application-autoscaling")

sagemaker_role = sagemaker.get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [8]:
def read_glue_table(database_name, table_name):
    # Create Glue client
    glue_client = boto3.client('glue')

    # Get the Glue table metadata
    response = glue_client.get_table(DatabaseName=database_name, Name=table_name)
    location = response['Table']['StorageDescriptor']['Location']

    # Read data from the Glue table into a Pandas DataFrame
    try:
        df = pd.read_parquet(location)
        if df.empty:
            raise ValueError("CSV file is empty")
        return df
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return None

In [9]:
df_nl_tutor = read_glue_table('algorithm_search', 'nl_tutor')
df_nl_student = read_glue_table('algorithm_search', 'nl_student')
df_nl_relations = read_glue_table('algorithm_search', 'student_tutor')

df = df_nl_relations.merge(df_nl_tutor, how='left', on=['tutor'])
df = df.rename(columns={'registration_date': 'tutor_registration_date', 'city': 'tutor_city', 'heard_about': 'tutor_heard_about','latitude': 'tutor_latitude',
                       'longitude': 'tutor_longitude'}) 
df = df.merge(df_nl_student, how='left', left_on='student', right_on='student_id')
df = df.rename(columns={'city': 'student_city', 'heard_about': 'student_heard_about',
                        'latitude': 'student_latitude', 'longitude': 'student_longitude'})
df = df.drop(columns = {'last_active_at_x', 'country_x', 'state_x', 'last_active_at_y', 'country_y', 'state_y', 'student_id', 'registration_date'})

In [10]:
df['student'] = df['student'].astype('str')

df['tutor'] = df['tutor'].astype('str')

df['course'] = df['course'].astype('str')

df['matched'] = df['matched'].astype('bool')

df['tutor_city'] = df['tutor_city'].astype('str')

df['tutor_accepts_new_students'] = df['tutor_accepts_new_students'].astype('bool')

df['tutor_can_teach_online'] = df['tutor_can_teach_online'].astype('bool')

df['tutor_has_availability'] = df['tutor_has_availability'].astype('bool')

df['tutor_course_offerings_count'] = pd.to_numeric(df['tutor_course_offerings_count'], errors='coerce')
zero_count_rows = df[df['tutor_course_offerings_count'].isna()]
count_courses_by_tutor = df.groupby('tutor')['course'].nunique()
df.loc[zero_count_rows.index, 'tutor_course_offerings_count'] = zero_count_rows['tutor'].map(count_courses_by_tutor)
df['tutor_course_offerings_count'] = df['tutor_course_offerings_count'].astype('int64')

df['tutor_intake_date'] = pd.to_datetime(df['tutor_intake_date'])
df['tutor_intake_date'] = df['tutor_intake_date'].dt.tz_localize(None)

df['student_city'] = df['student_city'].astype('str')

df['has_online_lessons'] = df['has_online_lessons'].astype('category')

df['has_package'] = df['has_package'].astype('bool')

df['tutor_type'] = df['tutor_type'].astype('category')

df['previous_lesson'] = df['previous_lesson'].astype('bool')

In [11]:
def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = 6371 * c  # Radius of the Earth in kilometers

    return distance

#Only cases for that country
df.replace('nan', np.nan, inplace=True)
df = df.dropna(subset=['student_city', 'tutor_city'])
df = df.drop_duplicates()

#Distance
df['distance'] = np.where(df.apply(lambda row: haversine(row['student_latitude'], row['student_longitude'], row['tutor_latitude'], row['tutor_longitude']), axis=1) > df['tutor_max_travel_distance'], 0,
                         round(df.apply(lambda row: haversine(row['student_latitude'], row['student_longitude'], row['tutor_latitude'], row['tutor_longitude']), axis=1), 2))        

#Days since intake when matching
df['tutor_intake_date'] = pd.to_datetime(df['tutor_intake_date'])
df['tutor_days_since_intake'] = np.where(df['tutor_intake_date'].notnull(), (df['matching_date'] - df['tutor_intake_date']).dt.days, 10000)

#Fill missing value
df['student_heard_about'] = df['student_heard_about'].fillna("Not provided")
df['tutor_first_lesson_date'] = df['tutor_first_lesson_date'].fillna(df.matching_date)
df['tutor_manual_weight'] = df['tutor_manual_weight'].fillna(1.0)


#Process new variables
df['tutor_days_to_first_lesson'] = (df['tutor_first_lesson_date'] - df['tutor_registration_date']).dt.days

df['avg_lesson_per_student'] = np.where((df['tutor_total_accepted_lessons'] != 0) & (df['tutor_previous_number_of_students'] != 0), df['tutor_total_accepted_lessons'].astype(float) / df['tutor_previous_number_of_students'].astype(float), 1)

df['tutor_actual_cancellations'] = df['tutor_total_appointment_requests'] - df['tutor_total_accepted_lessons'] - df['tutor_cancellations']
df['tutor_actual_cancellations'] = df['tutor_actual_cancellations'].apply(lambda x: max(0, x))

df['lessons_per_relation'] = np.where(df['number_of_relations'] > 0, df['tutor_total_accepted_lessons'] / df['number_of_relations'], 0)

In [12]:
df = df[((df['tutor_total_appointment_requests'] != 0) & (df['tutor_average_duration'] != 0)) | ((df['tutor_total_appointment_requests'] == 0) & (df['tutor_average_duration'] == 0) & (df['tutor_previous_number_of_students'] == 0))]
df = df.dropna(subset=['has_online_lessons'])
df = df.loc[df.groupby(['student', 'tutor', 'course', 'matched'])['has_online_lessons'].idxmax()].reset_index(drop=True)
df = df.loc[df.groupby(['student', 'tutor', 'course', 'matched'])['matching_date'].idxmax()].reset_index(drop=True)

In [13]:
#multiplier to process the old ranking formula
df['first_multiplier_lessons'] = (np.where(df['tutor_total_accepted_lessons'] > 50, 2, 1)) * \
                                  (np.where(df['tutor_total_accepted_lessons'] > 70, 2, 1)) * \
                                  (np.where(df['tutor_total_accepted_lessons'] > 100, 2, 1)) * \
                                  (np.where(df['tutor_total_accepted_lessons'] > 140, 2, 1)) * \
                                  (np.where(df['tutor_total_accepted_lessons'] > 210, 2, 1)) * \
                                  (np.where(df['tutor_total_accepted_lessons'] > 280, 2, 1)) * \
                                  (np.where(df['tutor_total_accepted_lessons'] > 330, 2, 1)) * \
                                  (np.where(df['tutor_total_accepted_lessons'] > 400, 2, 1))

conditions = [
    (df['tutor_previous_number_of_students'] == 0) & (df['number_of_relations'] < 3),
    (df['tutor_previous_number_of_students'] == 1) & (df['tutor_total_accepted_lessons'] == 1),
    (df['tutor_previous_number_of_students'] != 0) 
]
choices = [
    4,
    1,
    (((df['tutor_total_accepted_lessons'] / df['tutor_previous_number_of_students']) ** 2) / (df['tutor_previous_number_of_students'] ** 2.2)) * df['first_multiplier_lessons']
]
default = 0

df['first_multiplier'] = np.select(conditions, choices, default)
df['first_multiplier'] = np.sqrt(df['first_multiplier'])

df['second_multiplier'] = np.where(df['number_of_relations'] > 0, (1 + df['tutor_previous_number_of_students'])/df['number_of_relations'], 1.4)
df['third_multiplier'] = np.where(df['tutor_has_availability'] == True, 1.2, 0.8)
df['fourth_multiplier'] = np.where((df['tutor_previous_number_of_students'] == 0) & (df['number_of_relations'] < 3) & (df['tutor_days_since_intake'] < 38), 8, 1)
df['offline_multiplier'] = 2.2 * np.where(df['tutor_type'] == 1, 1, 1.3) + 1

#Ranking score
df['tutor_manual_weight'] = df['tutor_manual_weight'].fillna(1)
df['online_rank_score'] = df['first_multiplier'] * df['second_multiplier'] * df['third_multiplier'] * df['fourth_multiplier'] * df['tutor_manual_weight']
df['offline_rank_score'] = df['first_multiplier'] * df['second_multiplier'] * df['third_multiplier'] * df['fourth_multiplier'] * df['tutor_manual_weight']/df['offline_multiplier']
df['old_formula'] = np.where(df['has_online_lessons'] != 1, df['offline_rank_score'], df['online_rank_score']) 

#Drop unecessary columns
df = df.drop(columns=['tutor_registration_date', 'tutor_first_lesson_date', 'matching_date', 'student_longitude', 'student_latitude', 'tutor_latitude', 'tutor_longitude', 'tutor_intake_date', 'tutor_manual_weight', 'student_heard_about'])
df = df.drop(columns=['first_multiplier', 'first_multiplier_lessons', 'second_multiplier', 'third_multiplier', 'fourth_multiplier', 'offline_multiplier', 'online_rank_score', 'offline_rank_score', 'tutor_days_since_intake'])
df = df.drop(columns=['tutor_city', 'student_city', 'tutor_total_appointment_requests', 'student_previous_lessons', 'student_total_accepted_intro_lessons', 'has_online_lessons', 'has_package'])
df = df.drop(columns=['tutor_course_appointment_requests', 'student_total_intro_requests', 'tutor_course_previous_number_of_students', 'tutor_cancellations', 'number_of_relations', 'student_tutor_has_intro'])

In [14]:
instrumental_formula = 'matched ~ student_average_duration + created_deals + availabile_hours + online_lesson_rate:tutor_can_teach_online + tutor_accepts_new_students + tutor_max_travel_distance:distance + tutor_course_offerings_count + old_formula + tutor_type + previous_lesson'

probit_model = sm.GLM.from_formula(formula=instrumental_formula, data=df, family=sm.families.Binomial(link=sm.families.links.Probit()))
probit_result = probit_model.fit()
predicted_probabilities = probit_result.predict(df)
probit_lp = probit_result.fittedvalues
mills_ratio = np.exp(-0.5 * probit_lp ** 2) / (np.sqrt(2 * np.pi) * predicted_probabilities)
inverse_mills_ratio = 1 / mills_ratio
df['inverse_mills_ratio'] = inverse_mills_ratio

In [15]:
import sklearn
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV, GridSearchCV, StratifiedShuffleSplit, StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures, FunctionTransformer
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import PoissonRegressor, ElasticNet
from statsmodels.genmod.families import Poisson
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score, recall_score, accuracy_score
from sklearn.inspection import PartialDependenceDisplay
from xgboost import XGBRegressor, XGBClassifier
from xgboost import callback
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

In [16]:
###Uncomment to remove outliers###
##If you no longer want to run code without outliers, run the above code from the top again##

mean = np.mean(df['tutor_student_number_of_lessons'])
std_dev = 2.5 * np.std(df['tutor_student_number_of_lessons'])
df = df[df['tutor_student_number_of_lessons'].between(mean - std_dev, mean + std_dev)]

X = df.drop(columns=['tutor_student_number_of_lessons', 'student', 'tutor', 'course', 'matched'])
y = df['tutor_student_number_of_lessons']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 2503)
kf = KFold(n_splits=5, shuffle=True, random_state=1900561252)
folds = []
for train_index, val_index in kf.split(X_train):
    folds.append((train_index, val_index))

def poisson_loss(y_true, y_pred):
    return np.mean(y_pred - y_true * np.log(y_pred))

def rmse_scorer(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred)**2))

scorer = make_scorer(rmse_scorer, greater_is_better=False)

##For testing whether count data is better
#scorer = make_scorer(poisson_loss, greater_is_better = False)